In [1]:
import torch
import numpy as np
import pandas as pd
from os import listdir
import random
random.seed()   
import sklearn.mixture as mix
import talib
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from hmmlearn import hmm
np.random.seed(42)


mypath = '/media/sweerts/Data/trading/cash_bot/data/'
extension = 'h5'
onlyfiles = [f for f in listdir(mypath) if f.endswith('.' + extension)]

# Prepare training data set

In [2]:
masterdata = pd.read_hdf('/media/sweerts/Data/trading/cash_bot/data/' + onlyfiles[0], mode='r')
masterdata['RSI'] = talib.RSI(masterdata['close'],140).astype(float)
masterdata = masterdata.dropna()
for file in onlyfiles[1:]:
    data = pd.read_hdf('/media/sweerts/Data/trading/cash_bot/data/' + file, mode='r')
    data['RSI'] = talib.RSI(data['close'],140).astype(float)
    data = data.dropna()
    masterdata = masterdata.append(data)
    


# train model with optimal number of hidden states

In [3]:
X =  masterdata['RSI'].values.reshape(-1, 1)
model = hmm.GaussianHMM(n_components=2, covariance_type="full", n_iter=1000).fit(X)
hidden_states = model.predict(X)
########################################################################### HMM #####################################
masterdata['HMM_RSI'] = hidden_states

In [4]:
from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(model, 'HMM.pkl') 
# and later you can load it
model = joblib.load('HMM.pkl')

# Test model

In [5]:
from sklearn.externals import joblib
masterdata = pd.read_hdf('/media/sweerts/Data/trading/cash_bot/data/' + onlyfiles[10], mode='r')
masterdata['RSI'] = talib.RSI(masterdata['close'],140).astype(float)
masterdata = masterdata.dropna()
X =  masterdata['RSI'].values.reshape(-1, 1)
model = joblib.load('HMM.pkl')
hidden_states1 = model.predict(X)
hidden_states2 = model.predict(X)
hidden_states3 = model.predict(X)
print(sum(hidden_states1),sum(hidden_states2),sum(hidden_states3))

4803 4803 4803


In [ ]:
print(sum((hidden_states1==hidden_states2)-1))

In [6]:
print(hidden_states1)

[1 0 0 ... 1 1 1]
